## Experimenting visulalization libraries and techniques


In [6]:
!uv pip install langchain-experimental

Using Python 3.13.3 environment at: C:\code\AiAgents\Talk2SQL\backend\.venv
Resolved 47 packages in 5.42s
Installed 1 package in 256ms
 + langchain-experimental==0.3.4


In [20]:
from langchain_experimental.utilities import PythonREPL

repl = PythonREPL()
result= repl.run("""import matplotlib.pyplot as plt
import numpy as np
import io
import base64
import matplotlib
matplotlib.use('Agg')
x = np.linspace(0, 10, 100)
y = np.sin(x)
plt.figure()
plt.plot(x, y)
plt.xlabel("X-axis")
plt.ylabel("Y-axis")
plt.title("Simple Sine Wave Plot")
buf = io.BytesIO()
plt.savefig(buf, format='png', bbox_inches='tight')
plt.close()
png_bytes = buf.getvalue()
base64_string = base64.b64encode(png_bytes).decode('utf-8')
print(base64_string)""")



C:\Users\hp\AppData\Local\Temp\ipykernel_10356\280045093.py:4: RuntimeWarning: coroutine 'PyodideSandbox.execute' was never awaited
  result= repl.run("""import matplotlib.pyplot as plt


In [22]:
import base64
image_data = base64.b64decode(result)
with open("output_image.png", "wb") as image_file:
    image_file.write(image_data)

## creating node for the visualization agent 

In [ ]:
from langchain.agents import create_react_agent
from langchain_core.tools import tool
from langchain_core.prompts import PromptTemplate
from langchain_experimental.utilities import PythonREPL


class VisualizationAgent:
    def __init__(self, llm):
        self.llm = llm
        self.repl = PythonREPL()
        self.code = None 

    def visualization_agent(self, query, query_execution_results)-> str:
        """Creates a visualization agent that generates visualizations based on query results.

            Args:
                llm: The language model instance.
            query: The user's query.
            query_execution_results: The results obtained from executing the query.

        Returns:
            A string representation of the generated visualization.
        """
        # Implement the visualization logic here
        prompt = f"""Generate a visualization for the following query results: {query_execution_results} based on the query: {query}.
        Use appropriate libraries like matplotlib or seaborn to create the visualization.
        Ensure the visualization is clear and informative.
        always return the visualization as a base64 encoded string.
        Example generated code :
            import matplotlib.pyplot as plt
            import numpy as np
            import io
            import base64
            import matplotlib
            matplotlib.use('Agg')
            x = np.linspace(0, 10, 100)
            y = np.sin(x)
            plt.figure()
            plt.plot(x, y)
            plt.xlabel("X-axis")
            plt.ylabel("Y-axis")
            plt.title("Simple Sine Wave Plot")
            buf = io.BytesIO()
            plt.savefig(buf, format='png', bbox_inches='tight')
            plt.close()
            png_bytes = buf.getvalue()
            base64_string = base64.b64encode(png_bytes).decode('utf-8')
            print(base64_string)
        """
        prompt_template=PromptTemplate.from_template(prompt)
        # Create the chain: PromptTemplate -> LLM
        llm_chain = prompt_template | self.llm
        response = llm_chain.invoke({
            "query": query,
            "query_execution_results": query_execution_results
        })

        self.code = response.content.removeprefix("```python").removesuffix("```")
        result = self.repl.run(self.code)
        return result
        
        
     

    



In [25]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite", google_api_key=os.environ["GEMINI_API_KEY"])



In [60]:
object = VisualizationAgent(llm)
response = object.visualization_agent("Create a sine wave plot", "The query results show a sine wave pattern.")


In [57]:
response.removeprefix("```python").removesuffix("```")

'\nimport matplotlib.pyplot as plt\nimport numpy as np\nimport io\nimport base64\nimport matplotlib\n\n# Use a non-interactive backend for generating plots in environments without a display\nmatplotlib.use(\'Agg\')\n\n# Generate data for a sine wave\nx = np.linspace(0, 4 * np.pi, 200)  # Two full cycles of the sine wave\ny = np.sin(x)\n\n# Create the plot\nplt.figure(figsize=(10, 6))  # Set a reasonable figure size\nplt.plot(x, y, label=\'sin(x)\', color=\'blue\')\n\n# Add labels and title for clarity\nplt.xlabel("X-axis (Radians)", fontsize=12)\nplt.ylabel("Y-axis", fontsize=12)\nplt.title("Sine Wave Visualization", fontsize=16)\nplt.grid(True, linestyle=\'--\', alpha=0.6)  # Add a grid for better readability\nplt.legend(fontsize=10)\n\n# Improve layout and save to a buffer\nbuf = io.BytesIO()\nplt.savefig(buf, format=\'png\', bbox_inches=\'tight\')\nplt.close()  # Close the plot to free up memory\n\n# Encode the plot as a base64 string\npng_bytes = buf.getvalue()\nbase64_string = bas

In [62]:
result = object.execute_code()

In [63]:
import base64
image_data = base64.b64decode(result)
with open("output_image.png", "wb") as image_file:
    image_file.write(image_data)